In [1]:
import numpy as np
import SimpleITK as sitk
import pandas as pd
from scipy import spatial

import fnmatch
import os
import re
import itertools
from ipywidgets import interact, interactive
from ipywidgets import widgets
from IPython.display import display

import matplotlib.pyplot as plt
%matplotlib inline

import time
#import numba as nb
#from numba import cuda

from sklearn import cluster

In [2]:
def read_image(file1,):
    image1 = sitk.GetImageFromArray(skimage.io.imread(file1))
    
    #downimage1 = downsample(image1,downscale)
    
    return sitk.RescaleIntensity(sitk.Cast(image1,sitk.sitkUInt8))

In [34]:
def getSpatialGraphPoints( spatial_graph, pre_cell_ind, post_cell_ind):
    with open(spatial_graph, 'r') as csb:
        edge_ids = []
        edge_num_points = []
        edge_point_coords = []

        looking_for_axon_id = False
        looking_for_dend_id = False
        looking_for_soma_id = False
        looking_for_num_edge_points = False
        looking_for_edge_labels = False
        looking_for_edge_coords = False

        axon_id = 0
        dend_id = 0
        soma_id = 0
        edge_num_points_id = 0
        edge_ids_id = 0
        edge_points_id = 0

        lines = csb.readlines()

        # for each manual landmark
        for line in lines:

            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print translation

            if not line.find("Cell_{}_Axon ".format(pre_cell_ind)) == -1:
                looking_for_axon_id = True
            if looking_for_axon_id and line.rfind("Id")>-1:
                axon_id = int(line[line.rfind("Id")+3:]) 
                looking_for_axon_id = False
                #print axon_id

            if not line.find("Cell_{}_Dendrite ".format(post_cell_ind)) == -1:
                looking_for_dend_id = True
            if looking_for_dend_id and line.rfind("Id")>-1:
                dend_id = int(line[line.rfind("Id")+3:]) 
                looking_for_dend_id = False
                #print dend_id
                
            if not line.find("Cell_{}_Soma ".format(post_cell_ind)) == -1:
                looking_for_soma_id = True
            if looking_for_soma_id and line.rfind("Id")>-1:
                soma_id = int(line[line.rfind("Id")+3:]) 
                looking_for_soma_id = False

            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("EDGE { int EdgeLabels } @")>-1:
                edge_ids_id = int(line[line.rfind("EDGE { int EdgeLabels } @")+len("EDGE { int EdgeLabels } @"):]) 
                #print edge_ids_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_points_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_points_id

            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            if looking_for_num_edge_points :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))

            if line.find("@{}".format(edge_ids_id)) == 0:
                looking_for_edge_labels = True
                #print "found {}".format(edge_ids_id)
                continue
            if looking_for_edge_labels :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_labels = False
                    continue
                else:
                    edge_ids.append( int(line ))

            if line.find("@{}".format(edge_points_id)) == 0:
                looking_for_edge_coords = True
                #print "found {}".format(edge_points_id)
                continue
            if looking_for_edge_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_coords = False
                    continue
                else:
                    edge_point_coords.append( map(float,line.split()))


    axon_points = []
    dend_points = []
    soma_points = []
    start_ind = 0
    end_ind = 0
    for i in range(len(edge_num_points)):
        start_ind = end_ind
        end_ind = start_ind + edge_num_points[i]
        if edge_ids[i] == axon_id:
            #print start_ind
            axon_points.append(edge_point_coords[start_ind:end_ind])
        if edge_ids[i] == dend_id:
            #print start_ind
            dend_points.append(edge_point_coords[start_ind:end_ind])
        if edge_ids[i] == soma_id:
            #print start_ind
            soma_points.append(edge_point_coords[start_ind:end_ind])

    #print axon_points,dend_points,soma_points
    return (list(itertools.chain.from_iterable(axon_points))),(list(itertools.chain.from_iterable(dend_points))),(list(itertools.chain.from_iterable(soma_points)))
    #return list([axon_points,dend_points,soma_points])

In [1]:
def getSpatialGraphPointsForLabel( spatial_graph_file, Label=0):
    with open(spatial_graph_file, 'r') as sp_gr:
        edge_ids = []
        edge_num_points = []
        edge_point_coords = []

        looking_for_axon_id = False
        looking_for_dend_id = False
        looking_for_soma_id = False
        looking_for_num_edge_points = False
        looking_for_edge_labels = False
        looking_for_edge_coords = False

        axon_id = 0
        dend_id = 0
        soma_id = 0
        edge_num_points_id = 0
        edge_ids_id = 0
        edge_points_id = 0

        lines = sp_gr.readlines()

        # for each manual landmark
        for line in lines:

            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print translation

            if not line.find(label) == -1:
                looking_for_axon_id = True
            if looking_for_axon_id and line.rfind("Id")>-1:
                axon_id = int(line[line.rfind("Id")+3:]) 
                looking_for_axon_id = False
                #print axon_id
            

            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("EDGE { int EdgeLabels } @")>-1:
                edge_ids_id = int(line[line.rfind("EDGE { int EdgeLabels } @")+len("EDGE { int EdgeLabels } @"):]) 
                #print edge_ids_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_points_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_points_id

            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            if looking_for_num_edge_points :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))

            if line.find("@{}".format(edge_ids_id)) == 0:
                looking_for_edge_labels = True
                #print "found {}".format(edge_ids_id)
                continue
            if looking_for_edge_labels :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_labels = False
                    continue
                else:
                    edge_ids.append( int(line ))

            if line.find("@{}".format(edge_points_id)) == 0:
                looking_for_edge_coords = True
                #print "found {}".format(edge_points_id)
                continue
            if looking_for_edge_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_coords = False
                    continue
                else:
                    edge_point_coords.append( map(float,line.split()))


    axon_points = []
    dend_points = []
    soma_points = []
    start_ind = 0
    end_ind = 0
    for i in range(len(edge_num_points)):
        start_ind = end_ind
        end_ind = start_ind + edge_num_points[i]
        if edge_ids[i] == axon_id:
            #print start_ind
            axon_points.append(edge_point_coords[start_ind:end_ind])
        if edge_ids[i] == dend_id:
            #print start_ind
            dend_points.append(edge_point_coords[start_ind:end_ind])
        if edge_ids[i] == soma_id:
            #print start_ind
            soma_points.append(edge_point_coords[start_ind:end_ind])

    #print axon_points,dend_points,soma_points
    return (list(itertools.chain.from_iterable(axon_points))),(list(itertools.chain.from_iterable(dend_points))),(list(itertools.chain.from_iterable(soma_points)))
    #return list([axon_points,dend_points,soma_points])

In [11]:
def get_edge_list_from_edge_coords(edge_point_coords,edge_num_points):
    edge_list = []
    edge_pt_start_ind = 0
    for edge_id in range(len(edge_num_points)):
        edge = []
        edge.append(edge_point_coords[edge_pt_start_ind:edge_pt_start_ind+edge_num_points[edge_id]])
        edge_list.append(edge_point_coords[edge_pt_start_ind:edge_pt_start_ind+edge_num_points[edge_id]])
        #print(edge_point_coords[edge_pt_start_ind:edge_pt_start_ind+edge_num_points[edge_id]])
        edge_pt_start_ind = edge_pt_start_ind + edge_num_points[edge_id]
        
    return edge_list

In [12]:
def getSpatialGraphPointsWithoutLabel( spatial_graph_file, ):
    with open(spatial_graph_file, 'r') as sp_gr:
        vertex_coords = []
        edge_connectivity = []
        edge_num_points = []
        edge_point_coords = []
        
        looking_for_vertex_coords = False
        looking_for_edge_connectivity = False
        looking_for_num_edge_points  = False
        looking_for_edge_point_coords = False

        vertex_coords_id = 0
        edge_connectivity_id = 0
        edge_num_points_id = 0
        edge_point_coords_id = 0

        lines = sp_gr.readlines()

        # for each manual landmark
        for line in lines:

            # find the @tags
            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print transform

            if line.rfind("VERTEX { float[3] VertexCoordinates } @")>-1:
                vertex_coords_id = int(line[line.rfind("VERTEX { float[3] VertexCoordinates } @")+len("VERTEX { float[3] VertexCoordinates } @"):]) 
                #print edge_connectivity_id
                
            if line.rfind("EDGE { int[2] EdgeConnectivity } @")>-1:
                edge_connectivity_id = int(line[line.rfind("EDGE { int[2] EdgeConnectivity } @")+len("EDGE { int[2] EdgeConnectivity } @"):]) 
                #print edge_connectivity_id
                
            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_point_coords_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_point_coords_id

            # set flags to read coords etc  
            if line.find("@{}".format(vertex_coords_id)) == 0:
                looking_for_vertex_coords = True
                #print "found {}".format(edge_num_points_id)
                continue
                
            if line.find("@{}".format(edge_connectivity_id)) == 0:
                looking_for_edge_connectivity = True
                #print "found {}".format(edge_num_points_id)
                continue
                
            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            
            if line.find("@{}".format(edge_point_coords_id)) == 0:
                looking_for_edge_point_coords = True
                #print "found {}".format(edge_points_id)
                continue

            if looking_for_vertex_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_vertex_coords = False
                    continue
                else:
                    vertex_coords.append( list(map(float,line.split())))
                    
            if looking_for_edge_connectivity:
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_connectivity = False
                    continue
                else:
                    edge_connectivity.append( list(map(int,line.split())))
            
            if looking_for_num_edge_points  :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))
            
            if looking_for_edge_point_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_point_coords = False
                    continue
                else:
                    edge_point_coords.append( list(map(float,line.split())))
                    
        return vertex_coords,edge_connectivity,edge_num_points,edge_point_coords,get_edge_list_from_edge_coords(edge_point_coords,edge_num_points)
    

In [13]:
def get_contour_perimeter(contour):
    perimeter = 0
    for edge_id in range(len(contour)):
        perimeter = perimeter + spatial.distance.euclidean(contour[edge_id][0],contour[edge_id][len(contour[edge_id])-1])

    return perimeter

In [7]:
def getNeuronMorphologyPoints( spatial_graph):
    with open(spatial_graph, 'r') as csb:
        edge_ids = []
        edge_num_points = []
        edge_point_coords = []

        looking_for_axon_id = False
        looking_for_dend_id = False
        looking_for_apical_dend_id = False
        looking_for_basal_dend_id = False
        looking_for_soma_id = False
        looking_for_num_edge_points = False
        looking_for_edge_labels = False
        looking_for_edge_coords = False

        axon_id = 0
        dend_id = 0
        apical_dend_id = 0
        basal_dend_id = 0
        soma_id = 0
        edge_num_points_id = 0
        edge_ids_id = 0
        edge_points_id = 0

        lines = csb.readlines()

        # for each manual landmark
        for line in lines:
            #print(line)
            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print translation

            if not line.find("Axon ") == -1:
                looking_for_axon_id = True
            if looking_for_axon_id and line.rfind("Id")>-1:
                axon_id = 6#int(line[line.rfind("Id")+3:]) 
                looking_for_axon_id = False
                #print (axon_id)

            if not line.find("Dendrite ") == -1:
                looking_for_dend_id = True
            
            if looking_for_dend_id and line.find("ApicalDendrite ") == -1:
                looking_for_apical_dend_id = True
                
            if looking_for_dend_id and line.find("BasalDendrite ") == -1:
                looking_for_basal_dend_id = True
                
            if looking_for_apical_dend_id and line.rfind("Id")>-1:
                
                apical_dend_id = 4#int(line[line.rfind("Id")+3,:]) 
                looking_for_apical_dend_id = False
                #print (dend_id)
            
            if looking_for_basal_dend_id and line.rfind("Id")>-1:
                basal_dend_id = 5#int(line[line.rfind("Id")+3:]) 
                looking_for_basal_dend_id = False
                
            if not line.find("Soma ") == -1:
                looking_for_soma_id = True
            if looking_for_soma_id and line.rfind("Id")>-1:
                soma_id = 7#int(line[line.rfind("Id")+3:]) 
                looking_for_soma_id = False

            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("EDGE { int EdgeLabels } @")>-1:
                edge_ids_id = int(line[line.rfind("EDGE { int EdgeLabels } @")+len("EDGE { int EdgeLabels } @"):]) 
                #print edge_ids_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_points_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_points_id

            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            if looking_for_num_edge_points :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))

            if line.find("@{}".format(edge_ids_id)) == 0:
                looking_for_edge_labels = True
                #print "found {}".format(edge_ids_id)
                continue
            if looking_for_edge_labels :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_labels = False
                    continue
                else:
                    edge_ids.append( int(line ))

            if line.find("@{}".format(edge_points_id)) == 0:
                looking_for_edge_coords = True
                #print "found {}".format(edge_points_id)
                continue
            if looking_for_edge_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_coords = False
                    continue
                else:
                    edge_point_coords.append( list(map(float,line.split())))


    axon_points = []
    dend_points = []
    soma_points = []
    start_ind = 0
    end_ind = 0
#     for i in range(len(edge_num_points)):
#         print(i)
#         start_ind = end_ind
#         end_ind = start_ind + edge_num_points[i]
#         if edge_ids[i] == axon_id:
#             #print start_ind
#             axon_points.append(edge_point_coords[start_ind:end_ind])
#         if edge_ids[i] == apical_dend_id or edge_ids[i] == basal_dend_id:
#             #print start_ind
#             dend_points.append(edge_point_coords[start_ind:end_ind])
#         if edge_ids[i] == soma_id:
#             #print start_ind
#             soma_points.append(edge_point_coords[start_ind:end_ind])

    #print axon_points,dend_points,soma_points
#     return (list(itertools.chain.from_iterable(axon_points))),(list(itertools.chain.from_iterable(dend_points))),(list(itertools.chain.from_iterable(soma_points)))
    #return list([axon_points,dend_points,soma_points])
    return edge_point_coords

In [27]:
def getSectionPiaWMPoints( spatial_graph):
    with open(spatial_graph, 'r') as csb:
        edge_ids = []
        edge_num_points = []
        edge_point_coords = []

        looking_for_pia_id = False
        looking_for_wm_id = False
        
        looking_for_num_edge_points = False
        looking_for_edge_labels = False
        looking_for_edge_coords = False

        pia_id = 0
        wm_id = 0
        
        edge_num_points_id = 0
        edge_ids_id = 0
        edge_points_id = 0

        lines = csb.readlines()

        # for each manual landmark
        for line in lines:
            #print(line)
            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print translation

            if not line.find("Pia ") == -1:
                looking_for_pia_id = True
            if looking_for_pia_id and line.rfind("Id")>-1:
                pia_id = int(line[line.rfind("Id")+3:]) 
                looking_for_pia_id = False
                #print (pia_id)
            
            if not line.find("WhiteMatter ") == -1:
                looking_for_wm_id = True
                
            if looking_for_wm_id and line.rfind("Id")>-1:
                wm_id = int(line[line.rfind("Id")+3:]) 
                looking_for_wm_id = False
                #print (wm_id)
            

            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("EDGE { int GraphLabels } @")>-1:
                edge_ids_id = int(line[line.rfind("EDGE { int GraphLabels } @")+len("EDGE { int GraphLabels } @"):]) 
                #print edge_ids_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_points_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_points_id

            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            if looking_for_num_edge_points :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))

            if line.find("@{}".format(edge_ids_id)) == 0:
                looking_for_edge_labels = True
                #print "found {}".format(edge_ids_id)
                continue
            if looking_for_edge_labels :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_labels = False
                    continue
                else:
                    edge_ids.append( int(line ))

            if line.find("@{}".format(edge_points_id)) == 0:
                looking_for_edge_coords = True
                #print "found {}".format(edge_points_id)
                continue
            if looking_for_edge_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_coords = False
                    continue
                else:
                    edge_point_coords.append( list(map(float,line.split())))


    pia_points = []
    wm_points = []
    #bv_points = []
    start_ind = 0
    end_ind = 0
    for i in range(len(edge_num_points)):
        #print(i)
        start_ind = end_ind
        end_ind = start_ind + edge_num_points[i]
        if edge_ids[i] == pia_id:
            #print start_ind
            pia_points.append(edge_point_coords[start_ind:end_ind])
        if edge_ids[i] == wm_id :
            #print start_ind
            wm_points.append(edge_point_coords[start_ind:end_ind])
#         if edge_ids[i] == bv_id:
#             #print start_ind
#             bv_points.append(edge_point_coords[start_ind:end_ind])

    #print axon_points,dend_points,soma_points
    return [(list(itertools.chain.from_iterable(pia_points))),(list(itertools.chain.from_iterable(wm_points)))]
    #return list([axon_points,dend_points,soma_points])
    

In [5]:
def getSpatialGraphStackPoints( spatial_graph):
    with open(spatial_graph, 'r') as csb:
        edge_ids = []
        edge_num_points = []
        edge_point_coords = []

        looking_for_axon_id = False
        looking_for_dend_id = False
        looking_for_soma_id = False
        looking_for_num_edge_points = False
        looking_for_edge_labels = False
        looking_for_edge_coords = False

        axon_id = 0
        dend_id = 0
        soma_id = 0
        edge_num_points_id = 0
        edge_ids_id = 0
        edge_points_id = 0

        lines = csb.readlines()

        # for each manual landmark
        for line in lines:

            if not line.find("TransformationMatrix ") == -1:
                transform = (line[line.rfind("TransformationMatrix ")+len("TransformationMatrix "):-1])
                transform = map(float, transform.split(" ")[:])
                #print translation

            if line.rfind("EDGE { int NumEdgePoints } @")>-1:
                edge_num_points_id = int(line[line.rfind("EDGE { int NumEdgePoints } @")+len("EDGE { int NumEdgePoints } @"):]) 
                #print edge_num_points_id

            if line.rfind("EDGE { int EdgeLabels } @")>-1:
                edge_ids_id = int(line[line.rfind("EDGE { int EdgeLabels } @")+len("EDGE { int EdgeLabels } @"):]) 
                #print edge_ids_id

            if line.rfind("POINT { float[3] EdgePointCoordinates } @")>-1:
                edge_points_id = int(line[line.rfind("POINT { float[3] EdgePointCoordinates } @")+len("POINT { float[3] EdgePointCoordinates } @"):]) 
                #print edge_points_id

            if line.find("@{}".format(edge_num_points_id)) == 0:
                looking_for_num_edge_points = True
                #print "found {}".format(edge_num_points_id)
                continue
            if looking_for_num_edge_points :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_num_edge_points = False
                    continue
                else:
                    edge_num_points.append( int(line ))

            if line.find("@{}".format(edge_ids_id)) == 0:
                looking_for_edge_labels = True
                #print "found {}".format(edge_ids_id)
                continue
            if looking_for_edge_labels :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_labels = False
                    continue
                else:
                    edge_ids.append( int(line ))

            if line.find("@{}".format(edge_points_id)) == 0:
                looking_for_edge_coords = True
                #print "found {}".format(edge_points_id)
                continue
            if looking_for_edge_coords :
                if (line.rfind("@") == 0 or line.isspace()): 
                    looking_for_edge_coords = False
                    continue
                else:
                    edge_point_coords.append( [float(line.split()[0]),float(line.split()[1]),float(line.split()[2])] )


    

    #print axon_points,dend_points,soma_points
    #return (list(itertools.chain.from_iterable(edge_point_coords))),(list(itertools.chain.from_iterable(dend_points))),(list(itertools.chain.from_iterable(soma_points)))
    #return list([axon_points,dend_points,soma_points])
    return (edge_point_coords)

In [1]:
def getLandmarkCoords(filename):
    #print filename
    manual_landmark_list = []
    with open(filename, 'r') as csb:
        lines = csb.readlines()
        number = 0
        
        # for each manual landmark
        for line in lines:
            #print line
            min_dist = 1000000
            prox_line_ind = 1000000
            if line.startswith("@1"):
                number = 1
                continue
            if number == 1 and line.isspace() == False:
                #print line.split()
                manual_landmark_list.append(list(map(float,line.split())))
        
    return manual_landmark_list

In [4]:
def getTransformFromHxFile(filename):
    #print filename
    manual_landmark_list = []
    with open(filename, 'r') as csb:
        lines = csb.readlines()
        
        # for each manual landmark
        for line in lines:
            
            if not line.find("setTranslation ") == -1:
                translation = map(float,line.split(' ')[2:5])
                #print translation
            if not line.find("setTransform ") == -1:
                transform = map(float,line.split(' ')[2:2+16])
                #print transform

    return translation,transform
        


In [11]:
def applyTransform(points,transform,translate=[]):
    
    nda = np.array(points)#.reshape([np.array(points).shape[0],3])
    #print nda.shape
    
    if len(translate)>0:
        #translate_reshape = np.array(translate).reshape([1,3])
        #print translate_reshape.shape
        #translate_reshaped=np.append(translate_reshape, np.ones([1,1]), axis=1)
        #np.append(np.array(translate),np.ones([np.array(translate).shape[0],1]),axis=1)
        #print points
        #print translate_reshaped
        nda = nda + translate
        
    nda = np.append(nda,np.ones([nda.shape[0],1]),axis=1)
        
    transform = np.array(transform)
    transform = transform.reshape([4,4],order='F')
    #inverse = np.linalg.inv(transform)
    nda = np.transpose(np.matmul((transform),np.transpose(nda)))
    
    
   
    return nda[:,0:3]



In [5]:
def applyInverseTransform(points,transform,translate=[]):
    
    nda = np.array(points)#.reshape([np.array(points).shape[0],3])
    #print nda.shape
    nda = np.append(nda,np.ones([nda.shape[0],1]),axis=1)

    transform = np.array(transform)
    transform = transform.reshape([4,4],order='F')
    inverse = np.linalg.inv(transform)
    nda = np.transpose(np.matmul(inverse,np.transpose(nda)))

    if len(translate)>0:
        translate_reshape = np.array(translate).reshape([1,3])
        #print translate_reshape.shape
        translate_reshaped=np.append(translate_reshape, np.ones([1,1]), axis=1)
        #np.append(np.array(translate),np.ones([np.array(translate).shape[0],1]),axis=1)
        #print points
        #print translate_reshaped
        nda = nda - translate_reshaped
    return nda[:,0:3]



In [28]:
def scaleToImageCoords(points,XY_RESOLUTION=1,Z_RESOLUTION=1):
    _points = np.array(points,dtype=np.int64)
    _points[:,0] = list(map(round,(_points[:,0] / XY_RESOLUTION)))
    _points[:,1] = list(map(round,(_points[:,1] / XY_RESOLUTION)))
    _points[:,2] = list(map(round,(_points[:,2] / Z_RESOLUTION)))
   
    return _points



In [28]:
def scaleToImageCoords2D(points,XY_RESOLUTION=1,Z_RESOLUTION=1):
    _points = np.reshape(np.array(points,dtype=np.int64),[len(points),2])
    _points[:,0] = list(map(round,(_points[:,0] / XY_RESOLUTION)))
    _points[:,1] = list(map(round,(_points[:,1] / XY_RESOLUTION)))
    #_points[:,2] = list(map(round,(_points[:,2] / Z_RESOLUTION)))
   
    return _points



In [5]:
def scaleToPhysicalCoords(points):
    if len(points)==0:
        return
    _points = np.array(points,dtype=np.float32)
    _points[:,0] = points[:,0] * XY_RESOLUTION
    _points[:,1] = points[:,1] * XY_RESOLUTION
    _points[:,2] = points[:,2] * Z_RESOLUTION
   
    return _points



In [57]:
def image_3D_slice_viewer(slice_num=10):
    plt.imshow(sitk.GetArrayViewFromImage(image)[slice_num,:,:])

In [8]:
# Axon  Image based on the skeleton
def getAxonorDendtriteImage(segim,axon_dend_skeleton_int,axon_dend_skeleton,radius=0):
    axon_only_im = sitk.Image(segim.GetWidth(),segim.GetHeight(),segim.GetDepth(),segim.GetPixelID())
    axon_only_im.SetOrigin(segim.GetOrigin())
    objvalues =  ((sitk.GetArrayFromImage(segim) \
                   [[map(int,axon_dend_skeleton[:,2])], \
                    [map(int,axon_dend_skeleton[:,1])], \
                    [map(int,axon_dend_skeleton[:,0])]]))
    pixelvalueslist = map(float,objvalues[0])
    pixelvalueslistfinal = np.array(pixelvalueslist)[np.array(pixelvalueslist)>1]
    
    for pixelvalue in pixelvalueslistfinal:
        if pixelvalue > 1:
            ncc = sitk.NeighborhoodConnectedImageFilter()
            ncc.SetRadius(radius)
            ncc.SetLower(pixelvalue)
            ncc.SetUpper(pixelvalue)
            ncc.SetReplaceValue(255)
            ncc.SetSeedList(axon_dend_skeleton_int)
            #ncc.SetSeed([2,85,7])
            axon_only_im = sitk.Or(axon_only_im,sitk.Cast(ncc.Execute(segim),segim.GetPixelID()))

    return axon_only_im

In [30]:
# Axon  Image based on the skeleton
def getAxonorDendtriteImageAutoUpdate(segim,seeds,radius=0):
    ncc = sitk.NeighborhoodConnectedImageFilter()
    ncc.SetRadius(radius)
    ncc.SetLower(2.0)
    ncc.SetUpper(float(np.max(sitk.GetArrayFromImage(segim))))
    ncc.SetReplaceValue(255)
    ncc.SetSeedList(seeds)
    return (sitk.Cast(ncc.Execute(segim),segim.GetPixelID()))

In [17]:
# Axon  Image based on the skeleton
#@cuda.jit
def getAxonorDendtriteFromMaskImage(segim,skelim,min_pixel_value=1,radius=0):
    axon_only_im = sitk.Image(segim.GetWidth(),segim.GetHeight(),segim.GetDepth(),segim.GetPixelID())
    axon_only_im.SetOrigin(segim.GetOrigin())
    masked = sitk.Mask(segim,skelim)
    npmasked = sitk.GetArrayFromImage(masked)
    inds = (np.where(npmasked>1))
    iminds = ([map(int,inds[2]),map(int,inds[1]),map(int,inds[0])])
    seeds = list(np.transpose(iminds))
    
    pixelvalues = np.unique(sitk.GetArrayFromImage(masked)[sitk.GetArrayFromImage(masked)>1])
    
    for pixelvalue in map(float,pixelvalues):
        if pixelvalue > min_pixel_value:
            #print pixelvalue
            ncc = sitk.NeighborhoodConnectedImageFilter()
            ncc.SetRadius(radius)
            ncc.SetLower(pixelvalue)
            ncc.SetUpper(pixelvalue)
            ncc.SetReplaceValue(255)
            ncc.SetSeedList(seeds)
            #ncc.SetSeed([2,85,7])
            axon_only_im = sitk.Or(axon_only_im,sitk.Cast(ncc.Execute(segim),segim.GetPixelID()))

    return axon_only_im


#image = getAxonorDendtriteFromMaskImage(segim,mask)
#interact(image_3D_slice_viewer,slice_num=widgets.IntSlider(min=0,max=image.GetDepth()-1,step=1))

In [10]:
# Axon  Image based on the skeleton
#@cuda.jit
def getAxonorDendtriteFromMaskImageAutoUpdate(segim,skelim,min_pixel_value=1,radius=0):
    axon_only_im = sitk.Image(segim.GetWidth(),segim.GetHeight(),segim.GetDepth(),segim.GetPixelID())
    axon_only_im.SetOrigin(segim.GetOrigin())
    masked = sitk.Mask(segim,skelim)
    npmasked = sitk.GetArrayFromImage(masked)
    npseg = sitk.GetArrayFromImage(masked)
    inds = (np.where(npmasked>1))
    iminds = ([map(int,inds[2]),map(int,inds[1]),map(int,inds[0])])
    seeds = list(np.transpose(iminds))
    
    ncc = sitk.NeighborhoodConnectedImageFilter()
    ncc.SetRadius(radius)
    ncc.SetLower(2)
    ncc.SetUpper(float(np.max(npseg)))
    ncc.SetReplaceValue(255)
    ncc.SetSeedList(seeds)
    
    #axon_only_im = sitk.Or(axon_only_im,sitk.Cast(ncc.Execute(segim),segim.GetPixelID()))

    return sitk.Cast(ncc.Execute(segim),segim.GetPixelID())


#image = getAxonorDendtriteFromMaskImage(segim,mask)
#interact(image_3D_slice_viewer,slice_num=widgets.IntSlider(min=0,max=image.GetDepth()-1,step=1))

In [11]:
# Axon  Image based on the skeleton
def getAxonorDendtriteMinValue(im,axon_dend_skeleton):
    objvalues =  ((sitk.GetArrayFromImage(im) \
                   [[map(int,axon_dend_skeleton[:,2])], \
                    [map(int,axon_dend_skeleton[:,1])], \
                    [map(int,axon_dend_skeleton[:,0])]]))
    
    return [float(np.min(objvalues)),float(np.min(objvalues)),float(np.min(objvalues))]

In [12]:
# Axon  Image based on the skeleton
def getAxonorDendtriteMinValueFromMask(im,maskim):
    masked = sitk.Mask(segim,maskim)
    npmasked = sitk.GetArrayFromImage(masked)
    inds = (np.where(npmasked>1))
    iminds = ([map(int,inds[2]),map(int,inds[1]),map(int,inds[0])])
    objvalues = np.unique(sitk.GetArrayFromImage(masked)[sitk.GetArrayFromImage(masked)>1])
    
    return [float(np.min(objvalues)),float(np.min(objvalues)),float(np.min(objvalues))]

In [49]:
# Validate the contacts
# if spine contact, there must be a path of objects connecting axon to dend shaft through spines 
def getValidOverlaps(overlapspineim,segimwithoutaxonim,dendshaftimagestrict):
    overlapclusters = []
    overlapcentroids = [] 
    arr = sitk.GetArrayFromImage(overlapspineim)
    inds = np.transpose(np.where(arr>0) )
    
    if len(inds) > 0:
        #tree = cluster.spectral.kneighbors_graph(inds,n_neighbors=24,metric='euclidean')
        #ncomps,labels = cluster.hierarchical.connected_components(tree)
        #spclu = cluster.spectral.SpectralClustering()
        agg = cluster.AgglomerativeClustering(linkage='complete')
        labels = agg.fit_predict(inds)
        for grouplabel in np.unique(labels):
            group_inds = inds[np.where(labels==grouplabel)]
            seedlist = []
            for seed in group_inds:
                seedlist.append([int(seed[2]),int(seed[1]),int(seed[0])])
            blaim = sitk.Mask(getAxonorDendtriteImageAutoUpdate(segimwithoutaxonim,seedlist), dendshaftimagestrict)
            if sitk.GetArrayFromImage(blaim).max() > 1:
                overlapclusters.append(seedlist)
                overlapcentroids.append(np.array(seedlist).mean(axis=0))
            
    return list([overlapcentroids,overlapclusters])

In [91]:
def getOverlapCentroids(overlapim,numClusters):
    overlapcentroidslist = []
    arr = sitk.GetArrayFromImage(overlapim)
    inds = np.transpose(np.where(arr>0) )
    if inds.size > 0:
        overlapcentroids= (cluster.k_means(inds,numClusters))[0]
        for centroid in overlapcentroids:
            overlapcentroidslist.append( [centroid[2],centroid[1],centroid[0]])
    return overlapcentroidslist

In [49]:
# Validate the contacts
# if spine contact, there must be a path of objects connecting axon to dend shaft through spines 
def getValidOverlaps(overlapspineim,segimwithoutaxonim,dendshaftimagestrict):
    overlapclusters = []
    overlapcentroids = [] 
    arr = sitk.GetArrayFromImage(overlapspineim)
    inds = np.transpose(np.where(arr>0) )
    
    if len(inds) > 0:
        #tree = cluster.spectral.kneighbors_graph(inds,n_neighbors=24,metric='euclidean')
        #ncomps,labels = cluster.hierarchical.connected_components(tree)
        #spclu = cluster.spectral.SpectralClustering()
        agg = cluster.AgglomerativeClustering(linkage='complete')
        labels = agg.fit_predict(inds)
        for grouplabel in np.unique(labels):
            group_inds = inds[np.where(labels==grouplabel)]
            seedlist = []
            for seed in group_inds:
                seedlist.append([int(seed[2]),int(seed[1]),int(seed[0])])
            blaim = sitk.Mask(getAxonorDendtriteImageAutoUpdate(segimwithoutaxonim,seedlist), dendshaftimagestrict)
            if sitk.GetArrayFromImage(blaim).max() > 1:
                overlapclusters.append(seedlist)
                overlapcentroids.append(np.array(seedlist).mean(axis=0))
            
    return list([overlapcentroids,overlapclusters])

In [3]:
def convertPointToImage(pts,templateimg):
    arrim = sitk.Image(templateimg.GetWidth(),templateimg.GetHeight(),templateimg.GetDepth(),templateimg.GetPixelID())
    arr = sitk.GetArrayFromImage(arrim)
    if len(pts)>0:
        apts = np.array(pts)
        arr[apts[:,2],apts[:,1],apts[:,0]] = 255
    im = sitk.GetImageFromArray(arr)
    im.SetOrigin(templateimg.GetOrigin())
    return im
     

In [15]:
def convertPointsToImage(pts,templateimg):
    arrim = sitk.Image(templateimg.GetWidth(),templateimg.GetHeight(),templateimg.GetDepth(),templateimg.GetPixelID())
    arr = sitk.GetArrayFromImage(arrim)
    if len(pts)>0:
        apts = np.transpose([list(map(int,np.array(pts)[:,0])),list(map(int,np.array(pts)[:,1])),list(map(int,np.array(pts)[:,2]))])
        arr[apts[:,2],apts[:,1],apts[:,0]] = 255
    im = sitk.GetImageFromArray(arr)
    im.SetOrigin(templateimg.GetOrigin())
    return im
     

In [14]:
def convertPointsToImage2D(pts,templateimg):
    arrim = sitk.Image(templateimg.GetWidth(),templateimg.GetHeight(),templateimg.GetPixelID())
    arr = sitk.GetArrayFromImage(arrim)
    if len(pts)>0:
        apts = np.transpose([list(map(int,np.array(pts)[:,0])),list(map(int,np.array(pts)[:,1]))])
        arr[apts[:,1],apts[:,0]] = 255
    im = sitk.GetImageFromArray(arr)
    im.SetOrigin([templateimg.GetWidth(),templateimg.GetHeight(),0])
    return im
     

In [16]:
def convertPointsToPolydata(pts):
    points = vtk.vtkPoints()
    vertices = vtk.vtkCellArray()

    for i in range(len(pts)):
        #print(len(pts[i]))
        ids = points.InsertNextPoint(pts[i])

        vertices.InsertNextCell(1,[i])

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetVerts(vertices)
    
    return polydata

In [17]:
def convertPointsToPolydata2D(pts):
    points = vtk.vtkPoints()
    vertices = vtk.vtkCellArray()

    for i in range(len(pts)):
        #print(len(pts[i]))
        ids = points.InsertNextPoint([pts[i][0],pts[i][1],0])

        vertices.InsertNextCell(1,[i])

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetVerts(vertices)
    
    return polydata

In [17]:
def convertPointsListToPolydata2D(ptslist):
    points = vtk.vtkPoints()
    vertices = vtk.vtkCellArray()

    for pts in ptslist:
        for i in range(len(pts)):
            #print(len(pts[i]))
            ids = points.InsertNextPoint([pts[i][0],pts[i][1],0])

            vertices.InsertNextCell(1,[i])

    polydata = vtk.vtkPolyData()
    polydata.SetPoints(points)
    polydata.SetVerts(vertices)
    
    return polydata

In [ ]:
def convertPolydataToPoints(pdata):
    pt_list = []
    for i in range(pdata.GetNumberOfPoints()):
        pt_list.append(pdata.GetPoints().GetPoint(i))
        
    return pt_list

In [51]:
def cropPointsToImageBoundingBox(pts,croppedim):
    if len(pts)==0:
        return list([])
    pts = pts - map(int,croppedim.GetOrigin())
    centroid = [int(croppedim.GetWidth()/2.0),int(croppedim.GetHeight()/2.0),int(croppedim.GetDepth()/2.0)]
    uppercorner = centroid[0]+int(croppedim.GetWidth()/2.0), centroid[1]+int(croppedim.GetHeight()/2.0), centroid[2]+int(croppedim.GetDepth()/2.0)
    lowercorner = centroid[0]-int(croppedim.GetWidth()/2.0), centroid[1]-int(croppedim.GetHeight()/2.0), centroid[2]-int(croppedim.GetDepth()/2.0)
    croppedpts = []
    for pt in pts:
        if ((pt[0]<uppercorner[0] and pt[1]<uppercorner[1] and pt[2]<uppercorner[2])
            and (pt[0]>lowercorner[0] and pt[1]>lowercorner[1] and pt[2]>lowercorner[2])):
            croppedpts.append(pt)
            
    return croppedpts

In [24]:
def scale_pts(pts, xy_scaling = 1, z_coord = 0, z_offset = 0):
    scaled_pts = []
    for i in range(len(pts)):
        if (len(pts[0])==2):
            scaled_pts.append([pts[i][0]*xy_scaling,pts[i][1]*xy_scaling,z_coord])
        else:
            scaled_pts.append([pts[i][0]*xy_scaling,pts[i][1]*xy_scaling,z_coord])
    
    return scaled_pts

In [13]:
def convertCroppedImageToPhysicalCoords(pts,croppedim):
    
    return applyTransform(scaleToPhysicalCoords(np.array(pts) + croppedim.GetOrigin()),transform_img,translate_img)


In [16]:
def getMinMaxPts(pts):
    return list([np.min(np.array(pts),axis=0), np.max(np.array(pts),axis=0)])

In [19]:
def getZOrientation(pts,croppedim):
    [axonminptim , axonmaxptim] = (getMinMaxPts(pts))

    axonminpt = convertCroppedImageToPhysicalCoords(axonminptim.reshape([1,3]),croppedim)
    axonmaxpt = convertCroppedImageToPhysicalCoords(axonmaxptim.reshape([1,3]),croppedim)
    xz = np.tanh((axonmaxpt[:,2]-axonminpt[:,2])/float(axonmaxpt[:,0]-axonminpt[:,0])) * 180 / np.pi
    yz = np.tanh((axonmaxpt[:,2]-axonminpt[:,2])/float(axonmaxpt[:,1]-axonminpt[:,1])) * 180 / np.pi
    return list([xz,yz])

In [41]:
def getImageStats(im):
    stats = sitk.StatisticsImageFilter()
    stats.Execute(im)
    stats.GetMean()
    stats.GetMaximum()
    stats.GetMinimum()
    stats.GetSigma()
    
    return list([stats.GetMaximum(),stats.GetMinimum(),stats.GetMean(),stats.GetSigma()])

In [66]:
def show(image):
    interact(image_3D_slice_viewer,slice_num=widgets.IntSlider(min=0,max=image.GetDepth()-1,step=1))
    return image

In [25]:
def show2D(image):
    img = sitk.GetArrayViewFromImage(image)
    #gray = rgb2gray(img)    
    plt.imshow(img, cmap = plt.get_cmap('gray'))
    plt.show()

In [65]:
def showlabel(im,label):
    overlay = sitk.LabelOverlayImageFilter()
    overlay.SetOpacity(0.5)
    overlay.SetBackgroundValue(0)
    overlay.SetColormap([255,0,0])
    image = overlay.Execute(im,label)
    interact(image_3D_slice_viewer,slice_num=widgets.IntSlider(min=0,max=image.GetDepth()-1,step=1))
    return image

In [ ]:
def getGradientImage(im):
    grad = sitk.GradientMagnitudeRecursiveGaussianImageFilter()
    grad.SetSigma(1)
    gradim = grad.Execute(croppedim)
    return sitk.Cast(gradim,sitk.sitkUInt8)
    #gradimsharp = sitk.LaplacianSharpening(gradimUint8)



In [67]:
def getSegmentedImage(im,watershed_level):
    seg = sitk.MorphologicalWatershedImageFilter()
    seg.SetLevel(watershed_level)
    seg.SetMarkWatershedLine(False)
    segim = seg.Execute(getGradientImage(im))
    #segimsharp = seg.Execute(gradimsharp)
    return sitk.Cast(segim,sitk.sitkUInt8)

In [71]:
def getAxonOrDendriteSegmentedImageForGivenWatershedLevel(croppedim,croppedaxonim,level):    
    segim = getSegmentedImage(croppedim,level)
    #initialSkelImage = sitk.BinaryThreshold( sitk.SignedMaurerDistanceMap( croppeddendim ), -1000, 1 )
    axonimagestrict = getAxonorDendtriteFromMaskImage(segim,croppedaxonim,min_pixel_value=1,radius=0)#overlay.Execute(cropped_image,gradsegim)
    axonsegim = sitk.Mask(segim,axonimagestrict)
    startNumSkelPixels = np.sum((sitk.GetArrayFromImage(sitk.And(axonsegim,croppedaxonim)))>0)
    
    return list([axonsegim,startNumSkelPixels])

In [74]:
def getAxonOrDendriteSegmentedImage(croppedim,croppedaxonim):    
    [axonsegim,startNumSkelPixels] = getAxonOrDendriteSegmentedImageForGivenWatershedLevel(croppedim,croppedaxonim,1)
    for level in range(1,100):
        [tmpsegim,numSkelPixels] = getAxonOrDendriteSegmentedImageForGivenWatershedLevel(croppedim,croppedaxonim,level)
        if (numSkelPixels < startNumSkelPixels):
            break
        else:
            axonsegim = tmpsegim
            level = level+1

    return [axonsegim,level-1]

In [83]:
def getPaddedImage(img,radius=1):
    pad = sitk.BinaryDilateImageFilter()
    pad.SetForegroundValue(255)
    pad.SetKernelRadius(radius)
    #paddedim =pad.Execute(sitk.Cast(img,sitk.sitkUInt8))
    return pad.Execute(img)

In [84]:
def convertToBinary(im):
    return sitk.BinaryThreshold(im,1,255,255,0)

In [85]:
def getNumberOfOverlapClusters(overlapim):    
    cc = sitk.ConnectedComponentImageFilter()
    cc.SetFullyConnected(True)
    ccim = cc.Execute(overlapim)
    return cc.GetObjectCount()
    

In [1]:
def getAxonOrDendSegmentStats(croppedim,axonordendsegim,skelim):
    a = sitk.GetArrayFromImage(sitk.Mask(croppedim,convertToBinary(axonordendsegim)))
    volume = 0
    brightness = 0
    volume = volume + (a>0).sum()
    brightness = brightness + a.sum()
    radii,avgrad = getRadii(axonordendsegim,skelim)
    mean,std,snr = getSNR(croppedim,axonordendsegim)
    return list([brightness,mean,std,snr,volume,volume*XY_RESOLUTION*XY_RESOLUTION*Z_RESOLUTION, brightness/(volume*XY_RESOLUTION*XY_RESOLUTION*Z_RESOLUTION+1), radii, avgrad])

In [1]:
def getSNR(croppedim,axonordendsegim):
    a = sitk.GetArrayFromImage(sitk.Mask(croppedim,convertToBinary(axonordendsegim)))
    structure = a[a>0]
    signal = np.mean(structure)
    b = sitk.GetArrayFromImage(sitk.Mask(croppedim,sitk.Not(axonordendsegim)))
    background = b[b>0]
    noise = np.var(background)
    return list([np.mean(structure),np.std(structure),signal/noise])
    

In [9]:
def getBoutons(im,croppedim):    
    a = sitk.GetArrayFromImage(im)
    croppedimarr = sitk.GetArrayFromImage(croppedim)
    objsizes = []
    objvals = np.unique(a)
    for objval in objvals:
        if objval == 0:
            objsizes.append(list([0,0]))
        else :
            objsizes.append(list([(a == objval).sum(),(croppedimarr[a == objval]).sum()]))
    # cluster objs into bouton and non bouton objs based on obj size and brightness
    objszarr = np.array(objsizes).reshape([np.array(objsizes).shape[0],2])
    centroids, labels, bla = cluster.k_means((objszarr),2)
    if np.max((objszarr[labels>0])[:,0]) > np.max((objszarr[labels==0])[:,0]):
        boutonszs = (objszarr[labels>0])[:,0]
    else:
        boutonszs = (objszarr[labels==0])[:,0]
    
    #for each bouton add it to bouton mask image
    b = sitk.GetArrayFromImage(im)
    b[:,:,:] = 0
    
    for boutonsize in boutonszs:
        boutonval = objvals[(objszarr[:,0] == boutonsize)][0]
        
        b[a == boutonval] = boutonval

    boutonim = sitk.GetImageFromArray(b)
    boutonim.SetOrigin(im.GetOrigin())
    return boutonim

In [24]:
def getRadii(im,skelim):
    radii = []
    radsum = 0
    radavg = 0
    a = sitk.GetArrayFromImage(im)

    for objval in np.unique(a):
        if objval > 0:
            b = sitk.GetArrayFromImage(im)
            b[:,:,:] = 0
            b[a == objval] = 255
            bi = convertToBinary(sitk.GetImageFromArray(b))
            bi.SetOrigin(im.GetOrigin())
            objskel = (sitk.Mask(convertToBinary(skelim),(bi)))
            h =(sitk.GetArrayFromImage(objskel) > 0).sum()
            v = (b>0).sum()
            #print v,h
            for r in range(1,20):
                ve = np.pi*(r**2)*h
                #print ve
                #print r
                if ve > v:
                    break
                else:
                    r = r+1

            radsum = radsum + r-1
            radii.append((r-1))
    if len(radii) > 0:
        radavg = radsum/float(len(radii))
    return list([radii,radavg])

In [2]:
def writeLandmarkCoords(filename, landmarkList):
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(landmarkList)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")
        
        if len(landmarkList)!=0:
            for landmark in landmarkList:
                #print(landmark[0][0])
                file.write("{}".format(landmark[0]))
                file.write(" ")
                file.write("{}".format(landmark[1]))
                file.write(" ")
                file.write("{}".format(landmark[2]))
                file.write("\n")

In [2]:
def writeLandmarkCoords2D(filename, landmarkList):
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(landmarkList)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")
        
        if len(landmarkList)!=0:
            for landmark in landmarkList:
                #print(landmark[0][0])
                file.write("{}".format(landmark[0]))
                file.write(" ")
                file.write("{}".format(landmark[1]))
                file.write(" ")
                file.write("{}".format(0))
                file.write("\n")

In [2]:
def writeLandmarkCoordsFromLists(filename, landmarkLists):
    pt_list = []
    for landmarklist in landmarkLists:
        for landmark in landmarklist: 
            pt_list.append(landmark)
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(pt_list)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")
        
        #for landmarklist in landmarkLists:
        for landmark in pt_list:
            #print(landmark[0][0])
            file.write("{}".format(landmark[0]))
            file.write(" ")
            file.write("{}".format(landmark[1]))
            file.write(" ")
            file.write("{}".format(landmark[2]))
            file.write("\n")

In [20]:
def writeLandmarkCoordsFromListofLists(filename, landmarkListslist):
    pt_list = []
    for landmarklists in landmarkListslist:
        for landmarklist in landmarklists: 
            for landmark in landmarklist: 
                pt_list.append(landmark)
    with open(filename, 'w') as file:
        file.write("# Avizo 3D ASCII 2.0 \n")
        file.write("define Markers ")
        file.write("{}".format(len(pt_list)))
        file.write("\n")
        file.write("Parameters {\n")
        file.write("    NumSets 1,\n")
        file.write("    ContentType \"LandmarkSet\"\n")
        file.write("}\n")

        file.write("Markers { float[3] Coordinates } @1\n")

        file.write("# Data section follows\n")
        file.write("@1\n")
        
        #for landmarklist in landmarkLists:
        for landmark in pt_list:
            #print(landmark[0][0])
            file.write("{}".format(landmark[0]))
            file.write(" ")
            file.write("{}".format(landmark[1]))
            file.write(" ")
            file.write("{}".format(landmark[2]))
            file.write("\n")

In [19]:
def appendPolyData(surf1,surf2):
    appender = vtk.vtkAppendPolyData()
    appender.AddInputData(surf1)
    appender.AddInputData(surf2)
    appender.Update()
    surface = appender.GetOutput()

    return surface

In [18]:
def appendPolyData(surflist):
    appender = vtk.vtkAppendPolyData()
    for surf in surflist:
        appender.AddInputData(surf)
    appender.Update()
    surface = appender.GetOutput()

    return surface

In [ ]:
def apply_icp_transform_to_list(surfaces,icp):
    tr_surfaces = []
    for surface in surfaces:
        icpTransformFilter = vtk.vtkTransformPolyDataFilter()
        icpTransformFilter.SetInputData(surface)
        icpTransformFilter.SetTransform(icp)
        icpTransformFilter.Update()
        tr_surfaces.append(icpTransformFilter.GetOutput())
        
    return tr_surfaces

In [ ]:
def writeSurfaceMeshFile(surf,filename):
    writer = vtk.vtkPolyDataWriter()
    writer.SetInputData(surf)
    writer.SetFileName(filename)
    writer.Write()

In [ ]:
def writeSurfaceMeshFilesList(surfs,filenames):
    writer = vtk.vtkPolyDataWriter()
    for i in range(len(surfs)):
        writer.SetInputData(surfs[i])
        writer.SetFileName(filenames[i])
        writer.Write()

In [ ]:
def resample_contours(contours,res,downscale=1):
    resampled_conts_list = []
    for cnt in contours:
        resampled_cont_list = []
        for i in range(len(cnt)):
            if i == len(cnt)-1:
                pt1 = np.reshape(np.array(cnt)[i,:],[1,3])
                pt2 = np.reshape(np.array(cnt)[0,:],[1,3])
            else:
                pt1 = np.reshape(np.array(cnt)[i,:],[1,3])
                pt2 = np.reshape(np.array(cnt)[i+1,:],[1,3])
            
            dist = (distance.cdist(pt1,pt2))
            num_pts_inserted = int(dist / res)
            #print('num_pts_inserted: {}'.format(num_pts_inserted))
            m = (pt2[:,1]-pt2[:,0]) / (pt1[:,1]-pt1[:,0])

            #print('needed_{}'.format(num_pts_inserted))
            ptlist = []
            ptlist.append(cnt[i])
            if i == len(cnt)-1:
                ptlist.append(cnt[0])
            else:
                ptlist.append(cnt[i+1])
            ptlist = getMidLine(ptlist,num_pts_inserted)
            #print('added{}'.format(len(ptlist)))
            for pt in ptlist:
                resampled_cont_list.append([pt[0]/downscale,pt[1]/downscale,])
        resampled_conts_list.append(resampled_cont_list)
    return resampled_conts_list

In [ ]:
def getMidLine(ptlist,limit):
    newptlist = []
    #print(ptlist)
    if len(ptlist) < (limit/2):
        for i in range(len(ptlist)-1):
            newptlist.append(ptlist[i])
            newptlist.append([(ptlist[i][0]+ptlist[i+1][0])/2,(ptlist[i][1]+ptlist[i+1][1])/2])
        newptlist.append(ptlist[i+1])
        ptlist = newptlist
        #print(ptlist)
        return getMidLine(ptlist,limit)
        #return newptlist
    else:
        return (ptlist)

In [ ]:
def erode_image(im,radius):
    erode = sitk.BinaryErodeImageFilter()
    erode.SetKernelRadius(radius)
    erode.SetKernelType(sitk.sitkAnnulus)
    erode.SetBackgroundValue(0)
    erode.SetForegroundValue(255)
    eroded = erode.Execute(im)
    
    return eroded

In [ ]:
def dilate_image(im,radius):
    dilate = sitk.BinaryDilateImageFilter()
    dilate.SetKernelRadius(radius)
    dilate.SetKernelType(sitk.sitkAnnulus)
    dilate.SetBackgroundValue(0)
    dilate.SetForegroundValue(255)
    dilated = dilate.Execute(im)
    
    return dilated

In [ ]:
def convertContourCoordsIntoBinaryImage(cont,im,resample_res = 0):
    if resample_res != 0:
        [resampled_pia] = resample_contours([cont],resample_res)
    pia_imcords = scaleToImageCoords2D(resampled_pia,XY_RESOLUTION=XY_RES)
    pia_cont_im = convertPointsToImage2D(pia_imcords,im)
    pia_cont_im.SetOrigin(im.GetOrigin())
    pia_hole_filled = erode_image(sitk.BinaryFillhole(dilate_image(pia_cont_im,1),foregroundValue=255,fullyConnected=False),1)
    
    return pia_hole_filled

In [ ]:
def showlabel2D(im,label):
    overlay = sitk.LabelOverlayImageFilter()
    overlay.SetOpacity(0.5)
    overlay.SetBackgroundValue(0)
    overlay.SetColormap([255,0,0])
    image = overlay.Execute(im,label)
    img = sitk.GetArrayViewFromImage(image)
    #gray = rgb2gray(img)    
    plt.imshow(img, )#cmap = plt.get_cmap('gray'))
    plt.show()